## Introduction

The first step to learn Tensorflow is to understand its main key feature, the __"computational graph"__ approach. Basically, all Tensorflow codes contain two important parts:

__Part 1:__ building the __GRAPH__ which represents the data flow of the computations

__Part 2:__ running a __SESSION__ which executes the operations in the graph

In fact, TensorFlow separates the definition of computations from their execution. These two parts are explained in more details in the following sections. Before that, remember that the first step is to import the Tensorflow library!

In [6]:
import tensorflow as tf

This gives Python access to all of TensorFlow's classes, methods, and symbols. Using this command, TensorFlow library will be imported under the alias __tf__ so that later we can use it instead of typing the whole term __tensorflow__ each time.

__What is a Tensor?__
TensorFlow programs use a tensor data structure to represent all data. Tensor is a multi-dimensional array (0-D tensor: scalar, 1-D tensor: vector, 2-D tensor: matrix, and so on). So TensorFlow is simply referring to the flow of the Tensors in the Graph.
<img src="files/files/graph.gif">


## GRAPH
The biggest idea of all of the big ideas about Tensorflow is that numeric computation is expressed as a computational graph. In other words, the backbone of any Tensorflow program is going to be a __Graph__. As mentioned on the TensorFlow website, "A __computational graph__ (or graph in short) is a series of TensorFlow operations arranged into a graph of nodes".

So First let's see what does a node and operation mean?! The best way to explain it is by looking at a simple example. Suppose we want to write the code for function $f(x,y)=x^2y+y+2$. The Graph in TensorFlow will be something like:
<img src="files/files/sample_graph.png" width="500" height="1000" >

As it can be seen, the graph is composed of a series of nodes connected to each other by edges. Each __node__ in the graph is called __op__ (short for operation). So we'll have one node for each operation; either for operations on tensors (like math operations) or generating tensors (like variables and constants). Each node takes zero or more tensors as inputs and produces a tensor as an output.

Now Let's build a simple computational graph.


### Example 1:

Let's add two values, say a=2 and b=3, using TensorFlow. To do so, we need to call __tf.add()__. To see how it works and how many arguments it needs, simply Google it and check the first link (TensorFlow official documentation). The documentation says it can be used like __tf.add(x, y, name=None)__ where x and y are the values to be added together and __name__ is the operation name, i.e. the name associated to the addition node on the graph.

If we call the operation __"Add"__, the code will be as follows:

In [2]:
import tensorflow as tf
a = 2
b = 3
c = tf.add(a, b, name='Add')
print(c)

Tensor("Add:0", shape=(), dtype=int32)


The generated graph and variables are:

__*Note__: The graph is generated using __Tensorboard__ which is a visualization tool comes with TensorFlow. We'll learn more about it in the next tutorials.
<img src="files/files/ex1.png" width="800" height="1500">

This code creates two input nodes (for inputs a=2 and b=3) and one output node for the addition operation (named Add). As you see, when we print out the variable __c__ (i.e. the output Tensor of the addition operation), it prints out the Tensor information; its name (Add), shape (__()__ means scalar), and type (32-bit integer). However, It does not spit out the result (2+3=5). Why?!

To actually evaluate the nodes, we must run the computational graph within a __Session__. In simple words, the written code only generates the graph which only determines the expected sizes of Tensors and operations to be executed on them. However, it doesn't assign a numeric value to any of the Tensors. To assign these values and make them flow through the graph, we need to run a session.

Therefore a TensorFlow Graph is something like a function definition in Python. It __WILL NOT__ do any computation for you (just like a function definition will not have any execution result). It __ONLY__ defines computation operations.

## Session
To compute anything, a graph must be launched in a session. Technically speaking, session places the graph ops onto Devices, such as CPUs or GPUs, and provides methods to execute them. In our simple example, to run the graph and get the value for c:

In [3]:
sess = tf.Session()
print(sess.run(c))
sess.close()

5


This code creates a Session object (assigned to __sess__), and then (the second line) invokes its run method to run enough of the computational graph to evaluate __c__. This means that it only runs that part of the graph which is necessary to get the value of __c__ (in this simple example, it runs the whole graph). The last line closes the session. 

The following code does the same thing and is more commonly used. The only difference is that there is no need to close the session at the end as it gets closed automatically.

In [4]:
with tf.Session() as sess:
    print(sess.run(c))

5


Now let's look at the created graph one more time. Don't you see anything weird?
<img src="files/files/ex1_2.png" width="500" height="1000">
Exactly! What is x and y?! We didn't define any x or y variable!

Well... To explain the reason more clearly, let's make up two names; say __"Python-name"__ and __"TensorFlow-name"__. 

In an ideal Tensorflow case, __tf.add()__ receives two __Tensors__ with defined __"TensorFlow-name"__ as input (These names are separate from __Python-name__). For example, by writing $c = tf.add(a, b, name='Add')$, we're actually creating a variable (or Tensor) with __c__ as its Python-name and __Add__ as the TensorFlow-name. 

In the above code, we passed two Python variables (a=2 and b=3) which only have Python-names (a and b), but no TensorFlow-name. TensorFlow uses the TensorFlow-names for visualizing the graphs. Since a and b have no TensorFlow-name, it uses some default names, x and y. 

__*Note:__ This name mismatch can easily be solved by using tf.constant() for creating the input variables as Tensors instead of simply using Python variables (a=2, b=3). This is explained thoroughly in the next tutorial where we talk about TensorFlow DataTypes. 

For now, we'll continue using Python variables and change the Python variable names __a__ and __b__ into __x__ and __y__ to solve the name mismatch temporarily. 

Now let's look at a more complicated example.

### Example 2:
Creating a graph with multiple math operations

In [5]:
import tensorflow as tf
x = 2
y = 3
add_op = tf.add(x, y, name='Add')
mul_op = tf.multiply(x, y, name='Multiply')
pow_op = tf.pow(add_op, mul_op, name='Power')
useless_op = tf.multiply(x, add_op, name='Useless')

with tf.Session() as sess:
    pow_out, useless_out = sess.run([pow_op, useless_op])

The created graph and the defined variables (Tensors and Python variables) will be like:
<img src="files/files/ex2.png" width="1000" height="2000">

I called one of the operations useless_op because it's output is not used by other operations. Here I'm gonna explain a very __IMPORTANT__ point. Given this graph, if we fetch the __pow_op__ operation, it will first run the __add_op__ and __mul_op__ to get their output tensor and then run __pow_op__ on them to compute the required output value. In other words __useless_op__ will not be ran as it's output tensor is not used in running the __pow_op__ operation. 

__This is one of the advantages of defining a graph and running a session on it! It helps running only the required operations of the graph and skip the rest. This specially saves a significant amount of time for us when dealing with huge networks with hundreds and thousands of operations.__

In the above code, as you can see in the defined session, we're fetching the value of two tensors (i.e. output tensors of __pow_op__ and __useless_op__) at the same time. This will run the whole graph to get the required output tensors.

I hope this post have halped you understand the concept of __Graph__ and __Session__ in TensorFlow. Thanks for reading! If you have any question or doubt, feel free to leave a comment below. You can also send us feedback through the __contact us__ page.